# Contenido <a id='back'></a>
* [Introducción](#intro) 
* [Etapa 1. Exploración de los datos](#exploracion_datos)
     * [Conclusión intermedia](#conclusion_intermedia)
     *[Posibles razones por las que hay valores ausentes en los datos](#posibles_razones)
     *[Conclusiones](#conclusiones)
* [Etapa 2. Transformación de datos](#transformacion_datos)
* [Etapa 3. Trabajar con valores ausentes](#valores_ausentes)
    * [Restaurar valores ausentes en total_income](#restaurar_total_income)
    * [Restaurar valores ausentes en days_employed](#restaurar_days_employed)
* [Etapa 4. Clasificación de datos](#clasificacion_datos)
* [Etapa 5. Comprobación de hipótesis](#comprobacion_hipotesis)
    * [5.1. Hipótesis 1: ¿Existe una correlación entre tener hijos y pagar a tiempo?](#hipotesis_1) 
    * [5.2. Hipótesis 2: ¿Existe una correlación entre la situación familiar y el pago a tiempo?](#hipotesis_2)
    * [5.3. Hipótesis 3: ¿Existe una correlación entre el nivel de ingresos y el pago a tiempo?](#hipotesis_3)
    * [5.4. Hipótesis 4: ¿Cómo afecta el propósito del crédito a la tasa de incumplimiento?](#hipotesis_4)
* [Etapa 6. Conclusión general](#conclusion_general)


# Análisis del riesgo de incumplimiento de los prestatarios <a id='intro'></a>

Tu proyecto consiste en preparar un informe para la división de préstamos de un banco. Deberás averiguar si el estado civil y el número de hijos de un cliente tienen un impacto en el incumplimiento de pago de un préstamo. El banco ya tiene algunos datos sobre la solvencia crediticia de los clientes.

Tu informe se tendrá en cuenta al crear una **puntuación de crédito** para un cliente potencial. La **puntuación de crédito** se utiliza para evaluar la capacidad de un prestatario potencial para pagar su préstamo.

Este proyecto tiene como propósito evaluar los datos de los clientes para determinar si estas variables pueden tener un impacto importante en el incumplimiento de prestamos.
Las hipótesis que se van a evaluar son las siguientes:
¿Existe una correlación entre tener hijos y pagar a tiempo?
¿Existe una correlación entre la situación familiar y el pago a tiempo?
¿Existe una correlación entre el nivel de ingresos y el pago a tiempo?
¿Cómo afecta el propósito del crédito a la tasa de incumplimiento?


## Abre el archivo de datos y mira la información general. 


In [1]:
#Importación de librerias

import pandas as pd # Cargar todas las librerías
import numpy as np



In [ ]:
#Importación de datos

df = pd.read_csv('/datasets/credit_scoring_eng.csv') # Carga los datos

## Ejercicio 1. Exploración de datos <a id='exploracion_datos'></a>

**Descripción de los datos**
- `children` - el número de hijos en la familia
- `days_employed` - experiencia laboral en días
- `dob_years` - la edad del cliente en años
- `education` - la educación del cliente
- `education_id` - identificador de educación
- `family_status` - estado civil
- `family_status_id` - identificador de estado civil
- `gender` - género del cliente
- `income_type` - tipo de empleo
- `debt` - ¿había alguna deuda en el pago de un préstamo?
- `total_income` - ingreso mensual
- `purpose` - el propósito de obtener un préstamo


In [ ]:
df.info() # Vamos a ver cuántas filas y columnas tiene nuestro conjunto de datos



In [ ]:
df.head(10) # vamos a mostrar las primeras filas N


Al revisar la tabla se puede ver que la columna days_employed tiene valores negativos y cantidad de días que no tienen sentido ya que serian mas años de lo que se podrían trabajar en toda su vida y en la columna education hay valores duplicados implícitos, los cuales hay que solucionar en caso que se necesite agrupar estos datos. 

In [ ]:
df.info() # Obtener información sobre los datos


Tenemos valores ausentes solo en dos columnas,estas son las relacionadas con la información laboral: days_employed y total_income y aparentemente tienen 2174 valores nulos.  

In [ ]:
# Veamos la tabla filtrada con valores ausentes de la primera columna donde faltan datos
df['days_employed'].value_counts(dropna= False)


Aparentemente los valores ausentes en days_employed son simétricos y se demuestra que tiene 2174 valores nulos y tienen la forma esperada NaN por lo que podemos hacer calculos con estos datos. Se deben revisar mas detenidamente para verificar si tienen algún patrón en particular. 

In [ ]:
# Apliquemos múltiples condiciones para filtrar datos y veamos el número de filas en la tabla filtrada.
ausentes_days_employed= df[df['days_employed'].isnull()]
ausentes_days_employed.info()

**Conclusión intermedia** <a id='conclusion_intermedia'></a>

El número de filas de la tabla filtrada total_income_group coincide con el número de valores ausentes y podemos ver que days_employed y total_income están relacionadas, cuando un valor es nulo el valor del otro también lo es. 

Se va a calcular el porcentaje de los valores ausentes en comparación con el conjunto de datos para determinar que hacer con ellos.

In [ ]:
# Vamos a investigar a los clientes que no tienen datos sobre la característica identificada y la columna con los valores ausentes

num_filas_df= df['debt'].count() # con una de las columnas que no tiene valores ausentes determino el numero de filas total  
num_filas_ausentes= df['total_income'].isna().sum() # determino la cantidad de valores ausentes en'total_income' 
porcentaje_ausentes= num_filas_ausentes / num_filas_df  
print((f'El porcentaje de los valores ausentes en comparación con el conjunto de datos completo es del: {porcentaje_ausentes:.0%}'))
      

In [ ]:
# Comprobación de la distribución
df['total_income'].describe() 

Se puede observar que la diferencia entre el valor mínimo y máximo es muy grande y que existe una diferencia considerable entre la media y la mediana esto impulsado por la cantidad de valores atipicos que contiene la muestra como el valor máximo que está impulsando los promedios hacia arriba.

**Posibles razones por las que hay valores ausentes en los datos** <a id='posibles_razones'></a>

El único patrón que se puede observar de valores ausentes es que cuando falta days_employed también falta total_income esto podría ser debido a algún tipo de problema en el formulario, no es obligatorio completar esta información o no están a la vista por lo que el 10 % de las personas no la completan. Se recomienda revisar estos items en el formulario para determinar por que se generan esta cantidad de valores ausentes.


In [ ]:
# Comprobando la distribución en el conjunto de datos entero
df.describe() 


**Conclusión intermedia**

La distribución del conjunto de datos original es muy similar a la distribución de la tabla filtrada con gran diferencia entre los valores máximos y mínimos y entre la media y la mediana. También se pueden observar gran cantidad de errores en los datos que afectan los resultado de la distribución, entre los errores tenemos gran cantidad de valores negativos en la cantidad de hijos y en días trabajados así como valores extremos que a simple vista se pueden ver como errores pero en vista de que es un estudio de información crediticia es posible que estos valores tan altos se deban al conjunto de un grupo familiar. Se recomienda hacer una revisión del formulario y configurar para que no permita el ingreso de valores negativos ya que estos afectan considerablente el calculo de la distribución de los datos.


In [ ]:
# Comprueba otras razones y patrones que podrían llevar a valores ausentes
df_not_na= df.dropna() 
#distribución con valores ausentes de la variable 'income_type'
print(df['income_type'].value_counts(normalize=True))
print()
#distribución sin valores ausentes de la variable 'income_type'
print(df_not_na['income_type'].value_counts(normalize=True))
#distribución con valores ausentes de la variable 'debt'
print()
print(df['debt'].value_counts(normalize=True))
print()
#distribución sin valores ausentes de la variable 'debt'
print(df_not_na['debt'].value_counts(normalize=True))
print()
#distribución con valores ausentes de la variable 'gender'
print(df['gender'].value_counts(normalize=True))
print()
#distribución sin valores ausentes de la variable 'gender'
print(df_not_na['gender'].value_counts(normalize=True))
print()
#distribución con valores ausentes de la variable 'family_status'
print(df['family_status'].value_counts(normalize=True))
print()
#distribución sin valores ausentes de la variable 'family_status'
print(df_not_na['family_status'].value_counts(normalize=True))

**Conclusión intermedia** 

Considero que los valores ausentes pueden estar relacionados con la columna gender ya que se notan diferencias en la distribución con valores ausentes y sin valores ausentes quizas puede estar relacionado con que las personas del género femenino tienden a tener salarios mas bajos y por ello evitan completar esta sección. es muy importante que en el formulario de ingreso de información estos items se debería como obligatorios ya que son considerados importantes para el estudio crediticio del cliente, además el formulario permite el ingreso de valores negativos, otro aspecto a tomar en  cuenta es que se debe especificar si el prestamo es solicitado por una sola persona o por un grupo familiar de esa manera se podría hacer un análisis mas efectivo. 

In [ ]:
# Comprobación de otros patrones: explica cuáles

**Conclusiones** <a id='conclusiones'></a>
Se puede observar en el conjunto de datos que cuando falta la información de days_employed también falta total_income por lo que se debería hacer enfasis en que estos campos seas obligatorios, los valores ausentes presentan un patrón y pueden estar relacionados con la columna gender. Se va a proceder a clasificar y agrupar la información para determinar el valor adecuado para completar estos valores ausentes.

## Transformación de datos <a id='transformacion_datos'></a>

In [ ]:
# Veamos todos los valores en la columna de educación para verificar si será necesario corregir la ortografía y qué habrá que corregir exactamente
print(df['education'].value_counts())

In [ ]:
# Arregla los registros si es necesario
df['education_lowecase'] = df['education'].str.lower()

In [ ]:
# Comprobar todos los valores en la columna para asegurarnos de que los hayamos corregido
print(df['education_lowecase'].value_counts())


In [ ]:
# Veamos la distribución de los valores en la columna `children`
count_children= df.groupby('children').size()
print(count_children) 
porcentaje_prob_children=  ((47+76) / num_filas_df) 
print((f'El porcentaje de los valores problemáticos en comparación con el conjunto de datos completo es del: {porcentaje_prob_children:.1%}'))
df['children'].describe() 

Se observan valores extraños en los datos tales como que 47 pesonas tienen -1 hijos y 76 personas tienen 20 hijos debido a que es un número de personas considerable con esta cantidad tan grande de hijos intuyo que esto pueden ser errores de tipeo, aún así el porcentaje de valores problematicos corresponde al 1% lo cual es bajo por lo que considero que lo mejor es reemplazarlos por la mediana y ser muy cuidadosos con estos datos ya que es una variable de interés. 

In [ ]:
mediana_children = df['children'].median()
try:
    
    df['children'] = df['children'].replace({20: mediana_children, -1: mediana_children})  

except:
    print('No se encontró ningún número problemático')


In [ ]:
# Comprobar la columna `children` de nuevo para asegurarnos de que todo está arreglado
df['children'].describe() 


In [ ]:
# Encuentra datos problemáticos en `days_employed`, si existen, y calcula el porcentaje
count_days_employed= df.groupby('days_employed').size()
print(count_days_employed.tail(10)) 
df['days_employed'].describe() 

A simple vista se ve que la cantidad de valores problemáticos es alta ya que el valor mínimo, máximo, 25%, 50%, 75% son valores negativos esto podría ser por errores de tipeo, estos se van a reemplazar por el valor absoluto del número y también tenemos un valor máximo muy alto que esto podría ser generado como ya habia mencionado por la suma de un grupo familiar o bien porque fueron ingresados en otra escala, por ahora todos los valores mayores a 10000 que equivalen a unos 27 años Proximadamente se van a reemplazar por el valor NAN así no interfieren en el estudio. 

In [ ]:
# Aborda los valores problemáticos, si existen.
df['days_employed'] = abs((df['days_employed']))
df.loc[df.days_employed>10000,'days_employed']=''
df['days_employed']=pd.to_numeric(df['days_employed'], errors='ignore')


In [ ]:
# Comprueba el resultado - asegúrate de que esté arreglado
df.info()
df['days_employed'].describe() 


In [ ]:
# Revisa `dob_years` en busca de valores sospechosos y cuenta el porcentaje
count_dob_years= df.groupby('dob_years').size()
print(count_dob_years.head()) 
df['dob_years'].describe() 
porcentaje_dob_years= (101 / num_filas_df) 
print((f'El porcentaje de los valores problemáticos en comparación con el conjunto de datos completo es del: {porcentaje_dob_years:.1%}'))
      


Según lo observado el conjunto de datos tiene algunos valores problemáticos ya que ninguna persona que quiera solicitar un crédito puede tener 0 años, seguramente otro error de tipeo y no se completó esta información, estos errores no son significativos ya que solo representan el 0.5 % del conjunto total de los datos, estos se van a sustituir por el valor de la mediana. Se recomienda programar el campo de edad para que no permita continuar si el campo está vacio. 

In [ ]:
# Resuelve los problemas en la columna `dob_years`, si existen
df.loc[df.dob_years==0,'dob_years']=''
df['dob_years']=pd.to_numeric(df['dob_years'], errors='ignore')
median_dob_years= df['dob_years'].median()  
try:
    
    df['dob_years'] = df['dob_years'].fillna(median_dob_years)  

except:
    print('No se encontró ningún número problemático')


In [ ]:
# Comprueba el resultado - asegúrate de que esté arreglado
df['dob_years'].describe() 


In [ ]:
# Veamos los valores de la columna
df['family_status'].value_counts() 

In [ ]:
# Aborda los valores problemáticos en `family_status`, si existen, no se encontraron valores problemáticos.



In [ ]:
# Comprueba el resultado - asegúrate de que esté arreglado


In [ ]:
# Veamos los valores en la columna
df['gender'].value_counts() 

In [ ]:
# Comprueba el resultado - asegúrate de que esté arreglado

In [ ]:
# Veamos los valores en la columna
df['income_type'].value_counts() 

In [ ]:
# Aborda los valores problemáticos, si existen. No se encontraron valores problemáticos. 

In [ ]:
# Comprueba el resultado - asegúrate de que esté arreglado



No hay suficiente información para determinar si hay valores duplicados ya que no hay un identificador único para cada cliente como por ejemplo nombre o número de documento.

In [ ]:
# Comprobar los duplicados
print(df.duplicated().sum())
duplicated_df= df[df.duplicated()]

In [ ]:
df= df.drop_duplicates().reset_index(drop=True) # Aborda los duplicados

In [ ]:
# Última comprobación para ver si tenemos duplicados
print(df.duplicated().sum())

In [ ]:
# Comprueba el tamaño del conjunto de datos que tienes ahora, después de haber ejecutado estas primeras manipulaciones
df.info()

En este momento el conjunto de datos es mas pequeño, ya que se eliminaron los duplicados explícitos y los implícitos en la columna eduaction, se solucionaron los valores erróneos en las columnas children, days_employed y dob_years
por lo que tenemos una menor cantidad de valores problemáticos. 

# Trabajar con valores ausentes <a id='valores_ausentes'></a>

Se procede a realizar los diccionarios con los valores que tienen un Id identificador, para de esta manera facilitar la manipulación de estos datos en caso de necesitarse mas adelante.

In [ ]:
# Encuentra los diccionarios
diccionario_education= df.set_index('education_id')['education'].to_dict()
diccionario_family_status= df.set_index('family_status_id')['family_status'].to_dict() 
print(diccionario_education)
print()
print(diccionario_family_status)

### Restaurar valores ausentes en `total_income` <a id='restaurar_total_income'></a>

Las columnas que contienen valores ausentes son days_employed y total_income en ambos casos se van a agrupar los datos para determinar el valor mas adecuado para reemplazarlos, ya sea con la media o la mediana segun la clasificación más adecuada para cada caso.


In [ ]:
# Vamos a escribir una función que calcule la categoría de edad
df['dob_years'].value_counts()

def dob_years_group(dob_years):
    if dob_years <= 18:
        return 'NA'
    elif dob_years <=30:
        return 'Joven'
    elif dob_years <=65:
        return 'Adulto'
    else:
        return 'Retirado'
    

In [ ]:
# Prueba si la función funciona bien
print(dob_years_group(10))
print(dob_years_group(30))
print(dob_years_group(65))
print(dob_years_group(80))

In [ ]:
# Crear una nueva columna basada en la función
df['dob_years_group']= df['dob_years'].apply(dob_years_group)



In [ ]:
# Comprobar cómo los valores en la nueva columna
df.head(10)


Hay varios factores que pueden influir directamente en los ingresos persibidos; estos pueden ser la edad, el nivel educativo y también es muy  probable que los hombres tengan ingresos mayores que las mujeres por lo que es importante también evaluar esta distribución.

In [ ]:
# Crea una tabla sin valores ausentes y muestra algunas de sus filas para asegurarte de que se ve bien
df_sin_ausentes= df.dropna()
df_sin_ausentes.info()
df_sin_ausentes.head(10)

In [ ]:
# Examina los valores medios de los ingresos en función de los factores que identificaste
print(df_sin_ausentes.groupby('education_lowecase')['total_income'].mean())
print()
print(df_sin_ausentes.groupby('dob_years_group')['total_income'].mean())
print()
print(df_sin_ausentes.groupby('gender')['total_income'].mean())

In [ ]:
# Examina los valores medianos de los ingresos en función de los factores que identificaste
print(df_sin_ausentes.groupby('education_lowecase')['total_income'].median())
print()
print(df_sin_ausentes.groupby('dob_years_group')['total_income'].median())
print()
print(df_sin_ausentes.groupby('gender')['total_income'].median())




Se hizo la compraración entre varios factores que pueden definir los ingresos mensuales de las personas,tales como, el nivel de educación, la edad y el genero. 


Se decide utilizar la mediana en base a los datos de dob_years_group ya que el valor de la mediana se ve menos influenciada por los valores atípicos significativos.

In [ ]:
#  Escribe una función que usaremos para completar los valores ausentes

df['total_income']= df.groupby('dob_years_group')['total_income'].transform(lambda x: x.fillna(x.median()))


In [ ]:
# Comprueba si funciona
print(df['total_income'].isna().sum()) 
df.info()

In [ ]:
# Comprobar el número de entradas en las columnas

df.info()

###  Restaurar valores en `days_employed` <a id='restaurar_days_employed'></a>

In [ ]:
# Distribución de las medianas de `days_employed` en función de los parámetros identificados
df.groupby('dob_years_group')['days_employed'].median()

In [ ]:
# Distribución de las medias de `days_employed` en función de los parámetros identificados
df.groupby('dob_years_group')['days_employed'].mean()


Se va a utilizar la mediana ya que se tiene una gran cantidad de valores atipicos significativos la diferencia es grande entre la media y la mediana. 

In [ ]:
# Escribamos una función que calcule medias o medianas (dependiendo de tu decisión) según el parámetro identificado
df['days_employed']= df.groupby('dob_years_group')['days_employed'].transform(lambda x: x.fillna(x.median()))

In [ ]:
# Comprueba que la función funciona
print(df['days_employed'].isna().sum()) 

In [ ]:
# Comprueba las entradas en todas las columnas: asegúrate de que hayamos corregido todos los valores ausentes
df.info()

## Clasificación de datos <a id='clasificacion_datos'></a>


In [ ]:
# Muestra los valores de los datos seleccionados para la clasificación

print(df['children'].value_counts())
print()
print(df['family_status'].value_counts())


In [ ]:
# Comprobar los valores únicos
print(df['children'].unique())

print()

print(df['family_status'].unique())

Los grupos principales que se pueden identificar de la variable 'children' son: Con hijos y Sin hijos y de la variable 'family_status' En pareja y Sin pareja

In [ ]:
# Escribamos una función para clasificar los datos en función de temas comunes

def children_group(children):
    if children >= 1:
        return 'Con hijos'
    else:
        return 'Sin hijos'


In [ ]:
# Crea una columna con las categorías y cuenta los valores en ellas

df['children_group']= df['children'].apply(children_group)
df_pivot_children_group = df.pivot_table(index='children_group',values='debt',aggfunc=['count','sum','mean'])
# Calcular la tasa de incumplimiento en función del número de hijos
df_pivot_children_group



In [ ]:
# Revisar todos los datos numéricos en la columna seleccionada para la clasificación
df['total_income'].head(10)

In [ ]:
# Obtener estadísticas resumidas para la columna
df['total_income'].describe()


Los rangos que se utilizarán para la clasificación son los siguientes Bajo <= 17224.844000 que es el valor del percentil mas pequeño y Medio <= 31320.304000 que son todos los valores que están por debajo del percentil 75 que es el mas alto pero por encima del percentil 25 y Alto >31320.304000 todos los valores que se encuentran dentro del percentil 75 que corresponde a los valores con los ingresos mas altos. 

In [ ]:
# Crear una función para clasificar en diferentes grupos numéricos basándose en rangos

def total_income_group(total_income):
    if total_income <= 17224.844000:
        return 'Ingreso Bajo'
    elif total_income <= 31320.304000:
        return 'Ingreso Medio'
    elif total_income > 31320.304000:
        return 'Ingreso Alto'
    

In [ ]:
# Crear una columna con categorías
df['total_income_group']= df['total_income'].apply(total_income_group) 

In [ ]:
# Contar los valores de cada categoría para ver la distribución
print(df['total_income_group'].value_counts())

## Comprobación de las hipótesis <a id='comprobacion_hipotesis'></a>


**¿Existe una correlación entre tener hijos y pagar a tiempo?** <a id='hipotesis_1'></a>

In [ ]:
# Comprueba los datos sobre los hijos y los pagos puntuales

df_pivot_children = df.pivot_table(index='children',values='debt',aggfunc=['count','sum','mean'])
# Calcular la tasa de incumplimiento en función del número de hijos
df_pivot_children



**Conclusión**

La tasa de conversión resultó ser mayor para las personas con 4 hijos, seguido por 2 hijos, 1 hijo, 3 hijos, sin hijos ya la tasa mas bajas fue para los que tienen 5 hijos que solo tiene 9 registros. 

**¿Existe una correlación entre la situación familiar y el pago a tiempo?** <a id='hipotesis_2'></a>

In [ ]:
# Comprueba los datos del estado familiar y los pagos a tiempo
df_pivot_family_status= df.pivot_table(index='family_status',values='debt',aggfunc=['count','sum','mean'])
                                       
# Calcular la tasa de incumplimiento basada en el estado familiar
df_pivot_family_status




**Conclusión**

Según la tasa de conversión las personas unmarried tienen el porcentaje mas alto de incumplimiendo seguida por civil partnership, seguido de las personas married, divorced y por ultimo widow / widower, de igual manera la tasa de impago por categoría es baja.  

**¿Existe una correlación entre el nivel de ingresos y el pago a tiempo?** <a id='hipotesis_3'></a>

In [ ]:
# Comprueba los datos del nivel de ingresos y los pagos a tiempo

df_pivot_total_income_group= df.pivot_table(index='total_income_group',values='debt',aggfunc=['count','sum','mean'])

# Calcular la tasa de incumplimiento 
df_pivot_total_income_group

**Conclusión**

Según los datos la tasa de incumplimento es mucho mayor para las personas con ingresos medios, seguido por los que tienen ingresos bajos y por ultimo las personas con ingresos altos, de igual manera se concluye que la tasa de inclumplimiento es baja. 

**¿Cómo afecta el propósito del crédito a la tasa de incumplimiento?** <a id='hipotesis_4'></a>

In [ ]:
df['purpose'].sort_values().unique()
def replace_duplicate_debt(wrong_values,correct_values):
        df['purpose']= df['purpose'].replace(wrong_values,correct_values)
        
#Eliminando duplicados implícitos
replace_duplicate_debt(wrong_values= ['getting an education','to become educated','university education','supplementary education','getting higher education','going to university','profile education','to get a supplementary education'], correct_values = 'education')
replace_duplicate_debt(wrong_values= ['building a real estate','construction of own property'], correct_values = 'building a property')
replace_duplicate_debt(wrong_values= ['buy commercial real estate','buy residential real estate','buying property for renting out','housing','housing transactions','property','purchase of my own house','purchase of the house','purchase of the house for my family','real estate transactions','transactions with commercial real estate','transactions with my real estate'], correct_values = 'buy real estate')
replace_duplicate_debt(wrong_values= ['buying a second-hand car','buying my own car','car','car purchase','cars','second-hand car purchase','to buy a car','to own a car'], correct_values = 'purchase of a car')
replace_duplicate_debt(wrong_values= ['having a wedding','to have a wedding','wedding ceremony'], correct_values = 'wedding')


# Consulta los porcentajes de tasa de incumplimiento para cada propósito del crédito y analízalos

df_pivot_purpose= df.pivot_table(index='purpose',values='debt',aggfunc=['count','sum','mean'])

# Calcular la tasa de incumplimiento 
df_pivot_purpose



**Conclusión**

El riesgo mas elevado de incumplimiento lo tienen las personas que solicitan prestamos para compra de autos, seguido muy de cerca por educación, boda, construir una propiedad, comprar una propiedad y renovación del hogar con un porcentaje bastante mas bajo que los demás.

# Conclusión general <a id='conclusion_general'></a>

En líneas generales el conjunto de datos tenía una gran cantidad de errores que dificultaron el analisis de este estudio, completar los valores ausentes fue complejo ya que las dos columnas que tenian valores ausentes presentaban valores problemáticos tales como valores negativos y cifras muy altas que no corresponden a valores dentro de la realidad. 

Para completar los valores ausentes en la columna 'total_income' se agruparon los datos por la edad de los solicitantes en Joven, Adulto y Retirado y en vista que las medidas de distribución entre la media y la mediana tenía diferencias grandes se optó por utilizar la mediana que correspondía dependiendo del grupo por edad.

Para completar los valores ausentes en la columna 'days_employed' se delimitó el conjunto de datos para los valores < 10000 días que corresponden a unos 27 años de trabajo aproximadamete ya que los valores problemáticos estaban afectando considerablente las medidas de distribución y se agruparon los datos también por la edad, en base a esta distribución se procedió a reemplazar los valores ausentes con la mediana.

Segúin los datos esta gran cantidad de valores problemáticos y ausentes es posible que ocurrieron por errores de tipeo y datos con escalas diferentes y es necesario configurar los campos del formulario de ingreso de información para solucionar estos problemas, también es necesario configurar el campo de motivo del prestamo con opciones ya predefinidas y así evitar la gran catidad de duplicados implícitos que dificultan mucho el analisis de estos datos.  

**En base al estudio de los datos se puede concluir lo siguiente:**

1- Si agrupamos en dos grandes grupos de personas Con hijos y Sin hijos tenemos que la tasa de conversión entre las personas con hijos es del 0.092391 mayor que para las personas sin hijos que es del 0.075334.

2- Si hacemos el analisis por cantidad de hijos la tasa de conversión resultó ser mayor para las personas con 4 hijos, seguido por 2 hijos, 1 hijo, 3 hijos, sin hijos en ese orden y la tasa mas bajas fue para los que tienen 5 hijos pero que solo tiene 9 registros. 

2- Según la tasa de conversión las personas unmarried tienen el porcentaje mas alto de incumplimiendo 0.097509, seguida por civil partnership, seguido de las personas married, divorced y por ultimo widow / widower con 0.065693, de igual manera la tasa de impago por categoría es baja.

3- Las personas con ingresos medios tienen un porcentaje de riesgo bastante mayor que los que tienen ingresos bajos y altos, es posible que sea porque las personas con ingresos altos y bajos no solicitan tantos prestamos, estre los datos tenemos que las personas con ingresos altos fueron 5368 y con ingresos bajos tambien fueron 5368, mientras que los que tienen ingresos medios son 10735. 

4- El riesgo mas elevado de incumplimiento lo tienen las personas que solicitan prestamos para compra de autos con 0.093547, seguido muy de cerca por educación, boda, construir una propiedad, comprar una propiedad y renovación del hogar con un porcentaje bastante mas bajo que los demás 0.05766.

5- El porcentaje mas bajo de riesgo lo tiene el grupo que solicitó prestamo para housing renovation mientras que el mas alto es para los que tienen 4 hijos con un 0.097561 seguido muy de cerca por unmarried con 0.097509. 



<div class="alert alert-block alert-danger">
    
# Comentarios generales
    
<b>Comentario del revisor</b> <a class="tocSkip"></a>
    
Muy buen trabajo, Yaminir. Has corregido casi todo lo sugerido. Solo nos quedan tres puntos en los cuales trabajar para poder aprobar el proyecto. No te tomará mucho tiempo:
    
1. Tabla de contenidos al principio del documento.
2. Separación de importaciones.
3. Sustituir el valor de 0 por la mediana en `dob_years`.
</div>

<div class="alert alert-block alert-success">
    
# Comentarios generales
    
<b>Comentario del revisor</b> <a class="tocSkip"></a>
    
Gran trabajo, Yaminir. Has corregido todo lo sugerido y aprobado un nuevo proyecto.
    
¡Felicitaciones!
</div>